# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [127]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [128]:
stocks=pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [129]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data


{'companyName': 'Apple Inc',
 'marketcap': 2252379807618,
 'week52high': 187.31,
 'week52low': 132.98,
 'week52highSplitAdjustOnly': 191.65,
 'week52lowSplitAdjustOnly': 133.08,
 'week52change': 0.04309780389996949,
 'sharesOutstanding': 16797538019,
 'float': 0,
 'avg10Volume': 95416117,
 'avg30Volume': 87336045,
 'day200MovingAvg': 163.04,
 'day50MovingAvg': 143.88,
 'employees': 152037,
 'ttmEPS': 6.21,
 'ttmDividendRate': 0.9242629880789083,
 'dividendYield': 0.006684476105552532,
 'nextDividendDate': '',
 'exDividendDate': '2022-04-30',
 'nextEarningsDate': '2022-07-13',
 'peRatio': 22.86810274246768,
 'beta': 1.3200154047693562,
 'maxChangePercent': 53.427027343727,
 'year5ChangePercent': 3.0132833130412733,
 'year2ChangePercent': 0.5481206224592781,
 'year1ChangePercent': 0.0405053498475672,
 'ytdChangePercent': -0.21989769865144584,
 'month6ChangePercent': -0.22104925417974372,
 'month3ChangePercent': -0.18687300166129617,
 'month1ChangePercent': -0.03492638374109155,
 'day30Ch

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [130]:
data['month3ChangePercent']

-0.18687300166129617

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [131]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Three-Month Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [132]:
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

final_dataframe=pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url=f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe=final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A'

                ],
                index=my_columns
            ),ignore_index=True
        )
final_dataframe




/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_32125/1360647237.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe=final_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_32125/1360647237.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe=final_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_32125/1360647237.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe=final_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_32125/1360647237.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_datafr

,Ticker,Price,Three-Month Price Return,Number of Shares to Buy
0,A,121.4,-0.119809,N/A
1,AAL,14.027,-0.202401,N/A
2,AAP,186.51,-0.155611,N/A
3,AAPL,141.306,-0.190044,N/A
4,ABBV,156.14,-0.050919,N/A
...,...,...,...,...
496,YUM,114.6,-0.026979,N/A
497,ZBH,114.15,-0.120636,N/A
498,ZBRA,320.78,-0.306189,N/A
499,ZION,55.5,-0.251617,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [133]:
final_dataframe.sort_values('Three-Month Price Return',ascending=False,inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
# del final_dataframe['level_0']
final_dataframe

,index,Ticker,Price,Three-Month Price Return,Number of Shares to Buy
0,124,CXO,68.8,0.358513,N/A
1,225,HRB,36.48,0.349091,N/A
2,289,LW,73.8,0.269186,N/A
3,107,COG,23.13,0.259679,N/A
4,273,LB,83.23,0.224979,N/A
5,310,MNST,96.529,0.181198,N/A
6,314,MRK,96.14,0.165918,N/A
7,470,VRTX,294.269,0.146884,N/A
8,435,TIF,132.94,0.130122,N/A
9,259,K,70.68,0.129686,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [134]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

1000000


In [135]:
position_size=float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy']=math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

,index,Ticker,Price,Three-Month Price Return,Number of Shares to Buy
0,124,CXO,68.8,0.358513,290
1,225,HRB,36.48,0.349091,548
2,289,LW,73.8,0.269186,271
3,107,COG,23.13,0.259679,864
4,273,LB,83.23,0.224979,240
5,310,MNST,96.529,0.181198,207
6,314,MRK,96.14,0.165918,208
7,470,VRTX,294.269,0.146884,67
8,435,TIF,132.94,0.130122,150
9,259,K,70.68,0.129686,282


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [136]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_32125/1988529235.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_32125/1988529235.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_32125/1988529235.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_32125/1988529235.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe 

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.044,N/A,-0.199099,N/A,-0.256439,N/A,-0.119962,N/A,-0.060797,N/A,N/A
1,AAL,13.946,N/A,-0.423579,N/A,-0.295678,N/A,-0.203448,N/A,-0.235709,N/A,N/A
2,AAP,189.89,N/A,-0.087649,N/A,-0.227611,N/A,-0.158962,N/A,-0.017065,N/A,N/A
3,AAPL,145.59,N/A,0.04071,N/A,-0.214002,N/A,-0.190631,N/A,-0.033851,N/A,N/A
4,ABBV,153.8,N/A,0.378672,N/A,0.145454,N/A,-0.049492,N/A,0.009628,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,118.85,N/A,-0.023485,N/A,-0.169392,N/A,-0.02721,N/A,-0.008459,N/A,N/A
497,ZBH,111.35,N/A,-0.360412,N/A,-0.161664,N/A,-0.117748,N/A,-0.124395,N/A,N/A
498,ZBRA,315.02,N/A,-0.411492,N/A,-0.493504,N/A,-0.296266,N/A,-0.116784,N/A,N/A
499,ZION,53.87,N/A,-0.043576,N/A,-0.184846,N/A,-0.250275,N/A,-0.066231,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [137]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
        #  hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = 0

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.297405
1      0.055888
2      0.518962
3      0.718563
4       0.96008
         ...   
496    0.596806
497    0.103792
498    0.067864
499    0.570858
500    0.516966
Name: One-Year Return Percentile, Length: 501, dtype: object
0      0.301397
1      0.221557
2      0.377246
3      0.407186
4      0.924152
         ...   
496    0.483034
497     0.50499
498    0.025948
499    0.459082
500    0.199601
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.528942
1      0.255489
2      0.411178
3      0.297405
4      0.732535
         ...   
496    0.790419
497     0.53493
498    0.081836
499    0.145709
500     0.54491
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      0.493014
1      0.017964
2      0.706587
3      0.626747
4      0.850299
         ...   
496    0.746507
497    0.183633
498    0.197605
499     0.46507
500    0.918164
Name: One-Month Return Percentile, Length: 501, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.044,N/A,-0.199099,0.297405,-0.256439,0.301397,-0.119962,0.528942,-0.060797,0.493014,N/A
1,AAL,13.946,N/A,-0.423579,0.055888,-0.295678,0.221557,-0.203448,0.255489,-0.235709,0.017964,N/A
2,AAP,189.89,N/A,-0.087649,0.518962,-0.227611,0.377246,-0.158962,0.411178,-0.017065,0.706587,N/A
3,AAPL,145.59,N/A,0.04071,0.718563,-0.214002,0.407186,-0.190631,0.297405,-0.033851,0.626747,N/A
4,ABBV,153.8,N/A,0.378672,0.96008,0.145454,0.924152,-0.049492,0.732535,0.009628,0.850299,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,118.85,N/A,-0.023485,0.596806,-0.169392,0.483034,-0.02721,0.790419,-0.008459,0.746507,N/A
497,ZBH,111.35,N/A,-0.360412,0.103792,-0.161664,0.50499,-0.117748,0.53493,-0.124395,0.183633,N/A
498,ZBRA,315.02,N/A,-0.411492,0.067864,-0.493504,0.025948,-0.296266,0.081836,-0.116784,0.197605,N/A
499,ZION,53.87,N/A,-0.043576,0.570858,-0.184846,0.459082,-0.250275,0.145709,-0.066231,0.46507,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [164]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,81.22,246,2.354988,1.0,0.835954,0.998004,0.219587,0.992016,0.083226,0.966068,0.989022
1,HRB,35.54,562,0.484352,0.974052,0.494289,0.994012,0.339172,0.998004,0.042395,0.924152,0.972555
2,COG,22.46,890,0.341447,0.948104,0.189389,0.942116,0.257203,0.994012,0.250985,0.998004,0.970559
3,VRTX,298.399,67,0.514818,0.98004,0.270136,0.966068,0.149251,0.986028,0.057308,0.948104,0.97006
4,LLY,331.5,60,0.473025,0.97006,0.154288,0.926148,0.104278,0.972056,0.03395,0.912176,0.94511
5,DLTR,161.82,123,0.541517,0.984032,0.138621,0.918164,0.002404,0.854291,0.198788,0.996008,0.938124
6,AZO,2161.8,9,0.469731,0.968064,0.047938,0.836327,0.069241,0.944112,0.170278,0.99002,0.934631
7,DG,250.32,79,0.150311,0.864271,0.109389,0.898204,0.103366,0.968064,0.262244,1.0,0.932635
8,HSY,220.62,90,0.28901,0.936128,0.15907,0.93014,0.050175,0.93014,0.042516,0.926148,0.930639
9,BMY,80.657,247,0.234804,0.92016,0.283909,0.97006,0.09866,0.966068,0.01308,0.860279,0.929142


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [165]:
hqm_dataframe=hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:50]
try:
    del hqm_dataframe['level_0']
except:
    pass
hqm_dataframe.reset_index(inplace=True,drop=True)

try:
    del hqm_dataframe['level_0']
except:
    pass
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,81.22,246,2.354988,1.0,0.835954,0.998004,0.219587,0.992016,0.083226,0.966068,0.989022
1,HRB,35.54,562,0.484352,0.974052,0.494289,0.994012,0.339172,0.998004,0.042395,0.924152,0.972555
2,COG,22.46,890,0.341447,0.948104,0.189389,0.942116,0.257203,0.994012,0.250985,0.998004,0.970559
3,VRTX,298.399,67,0.514818,0.98004,0.270136,0.966068,0.149251,0.986028,0.057308,0.948104,0.97006
4,LLY,331.5,60,0.473025,0.97006,0.154288,0.926148,0.104278,0.972056,0.03395,0.912176,0.94511
5,DLTR,161.82,123,0.541517,0.984032,0.138621,0.918164,0.002404,0.854291,0.198788,0.996008,0.938124
6,AZO,2161.8,9,0.469731,0.968064,0.047938,0.836327,0.069241,0.944112,0.170278,0.99002,0.934631
7,DG,250.32,79,0.150311,0.864271,0.109389,0.898204,0.103366,0.968064,0.262244,1.0,0.932635
8,HSY,220.62,90,0.28901,0.936128,0.15907,0.93014,0.050175,0.93014,0.042516,0.926148,0.930639
9,BMY,80.657,247,0.234804,0.92016,0.283909,0.97006,0.09866,0.966068,0.01308,0.860279,0.929142


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [166]:
portfolio_input()

In [167]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,81.22,246,2.354988,1.0,0.835954,0.998004,0.219587,0.992016,0.083226,0.966068,0.989022
1,HRB,35.54,562,0.484352,0.974052,0.494289,0.994012,0.339172,0.998004,0.042395,0.924152,0.972555
2,COG,22.46,890,0.341447,0.948104,0.189389,0.942116,0.257203,0.994012,0.250985,0.998004,0.970559
3,VRTX,298.399,67,0.514818,0.98004,0.270136,0.966068,0.149251,0.986028,0.057308,0.948104,0.97006
4,LLY,331.5,60,0.473025,0.97006,0.154288,0.926148,0.104278,0.972056,0.03395,0.912176,0.94511
5,DLTR,161.82,123,0.541517,0.984032,0.138621,0.918164,0.002404,0.854291,0.198788,0.996008,0.938124
6,AZO,2161.8,9,0.469731,0.968064,0.047938,0.836327,0.069241,0.944112,0.170278,0.99002,0.934631
7,DG,250.32,79,0.150311,0.864271,0.109389,0.898204,0.103366,0.968064,0.262244,1.0,0.932635
8,HSY,220.62,90,0.28901,0.936128,0.15907,0.93014,0.050175,0.93014,0.042516,0.926148,0.930639
9,BMY,80.657,247,0.234804,0.92016,0.283909,0.97006,0.09866,0.966068,0.01308,0.860279,0.929142


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [160]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [161]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [162]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 30, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [163]:
writer.save()